In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os,sys,inspect
import gc
from tqdm import tqdm
import random

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from load import *
from evals import *
from models import WideAndDeep

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Embedding, Flatten, Multiply
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def leakyrelu(x, factor=0.2):
    return tf.maximum(x, factor*x)

In [3]:
df = load_data('../data/ml-100k/u.data', threshold=3)
uuid = df['userId'].unique()
uiid = df['movieId'].unique()


In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15, random_state=SEED, stratify=df['userId'].values)

In [5]:
tr_X = np.stack([train['userId'].values.astype(np.int32), train['movieId'].values.astype(np.int32)], 1)
test_X = np.stack([test['userId'].values.astype(np.int32), test['movieId'].values.astype(np.int32)], 1)

tr_X.shape, test_X.shape

((85000, 2), (15000, 2))

In [6]:
class CrossNetwork(layers.Layer):
    def __init__(self, n_layers):
        super(CrossNetwork, self).__init__()
        self.n_layers = n_layers
    
    def build(self, input_shape):
        dim = input_shape[-1]
        self.cross_weights = [self.add_weight(shape=(dim, 1), 
                                        initializer=tf.random_normal_initializer(),
                                       trainable=True,
                                       name=f'cross_weight_{i}') for i in range(self.n_layers)]
    
        self.cross_biases = [self.add_weight(shape=(dim, 1),
                                      initializer=tf.random_normal_initializer(),
                                      trainable=True,
                                      name=f'cross_bias_{i}') for i in range(self.n_layers)]
    def call(self, inputs):
        x_0 = tf.expand_dims(inputs, -1)
        x_l = x_0
        for i in range(self.n_layers):
            x_l1 = tf.tensordot(x_l, self.cross_weights[i], axes=[1, 0])
            x_l = tf.matmul(x_0, x_l1) + self.cross_biases[i]
            
        x_l = tf.squeeze(x_l, -1)
        
        return x_l

class DeepNetwork(layers.Layer):
    def __init__(self, units, activation='relu'):
        super(DeepNetwork, self).__init__()
        
        self.layers = [Dense(unit, activation=activation) for unit in units]
    
    def call(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
            
        return x

In [7]:
class DCN(Model):
    def __init__(self, x_dims, latent_dim, n_cross_layers, hidden_units, activation='relu', l2_emb=1e-4):
        super(DCN, self).__init__()
        self.x_dims = x_dims
        self.latent_dim = latent_dim
        
        self.cross_layers = CrossNetwork(n_cross_layers)
        self.deep_layers = DeepNetwork(hidden_units, activation)
        
        self.embedding = Embedding(sum(x_dims)+1, latent_dim, input_length=1, embeddings_regularizer=l2(l2_emb))
    
        self.flatten = Flatten()
        self.final_dense = Dense(1)
        
    def call(self, inputs):
        embed = self.embedding(inputs + tf.constant((0, *np.cumsum(self.x_dims)))[:-1])
        embed = self.flatten(embed)
        
        # if continuous, concat with embed
        
        cross_out = self.cross_layers(embed)
        deep_out = self.deep_layers(embed)
        
        out = tf.concat([cross_out, deep_out], 1)
        out = self.final_dense(out)
        
        return out

In [8]:
dcn = DCN((len(uuid), len(uiid)), 8, 2, [128,64])

In [9]:
dcn.compile(optimizer='adam',
           loss = losses.BinaryCrossentropy(from_logits=True))

dcn.fit(tr_X,
       train['rating'].values,
       epochs=10,
       shuffle=True)

Epoch 1/10
2657/2657 [==============================] - 9s 4ms/step - loss: 0.5938
Epoch 2/10
2657/2657 [==============================] - 9s 3ms/step - loss: 0.5610
Epoch 3/10
2657/2657 [==============================] - 9s 3ms/step - loss: 0.5538
Epoch 4/10
2657/2657 [==============================] - 9s 4ms/step - loss: 0.5485
Epoch 5/10
2657/2657 [==============================] - 9s 4ms/step - loss: 0.5420
Epoch 6/10
2657/2657 [==============================] - 9s 3ms/step - loss: 0.5362
Epoch 7/10
2657/2657 [==============================] - 9s 3ms/step - loss: 0.5313
Epoch 8/10
2657/2657 [==============================] - 9s 3ms/step - loss: 0.5264
Epoch 9/10
2657/2657 [==============================] - 9s 4ms/step - loss: 0.5207
Epoch 10/10
2657/2657 [==============================] - 9s 4ms/step - loss: 0.5157


In [10]:
pred = dcn.predict(test_X)


In [11]:
np.sum(np.where(pred>0., 1, 0).flatten() == test['rating'].values) / len(pred)

0.7228666666666667

In [12]:
from sklearn.metrics import precision_score, recall_score,  roc_auc_score, precision_recall_fscore_support

print(roc_auc_score(test['rating'].values, pred))
print(precision_score(test['rating'].values, np.where(pred>0., 1, 0)))
print(recall_score(test['rating'].values, np.where(pred>0., 1, 0)))

0.7906004666065357
0.7418904778514126
0.7671315220004808
